In [1]:
#importing nse library to get nse stock data

from nsepy import get_history
from datetime import date
import pandas as pd
import datetime


In [2]:
#reading the scrapped data 
money=pd.read_excel("E:\\allpraxis\\CAPP\\Scrapped Data1.xlsx")
money = money.dropna(subset = ['NSE CODE'])  #removing any blank fields
money = money.reset_index(drop=True) #avoiding old index to be added as a separate column

In [3]:
money.head()

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender
0,8144,"sep 15, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,sell,185,kotak securities
1,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay
2,9170,"jun 23, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,buy,195,icicidirect.com
3,9674,"may 14, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,152,sushil finance
4,9762,"may 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,reduce,143,krchoksey


In [4]:
#FOR EACH STOCK:-
# extracting start date(recommendation date).
# extracting target.
# extracting nearest value acheived, argmin(daily_closing_price - target).
# extracting date when nearest value acheived.
# days to acheive target(start date - acheived date)
money.shape

(5202, 9)

In [13]:
recommendation=money[1:2]
display(recommendation)

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender
1,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay


In [24]:
stock_code= recommendation['NSE CODE'].iloc[0]
stock_code

'PETRONET'

In [15]:
#Extracting date from extracted rows:
start_date= recommendation['Date'].iloc[0]
#start_date = start_date.drop([0])
#d = start_date[1]
#d
start_date

'  aug 06, 2014 '

In [12]:
#to get history of stock we have to pass date in date(%Y,%m,%d) format, thus converting:
#date_object = datetime.strptime(d, '%m %d %Y') # getting error because of ',' in date forat
#print(date_object)

In [16]:
#removing comma from date format:
#money['Date'].str.replace(',', '') # working
start_date = start_date.replace(',', '') #worked
start_date

'  aug 06 2014 '

In [17]:
#now converting date column in get_history formatdate(2018,8,20): 
#first modifying the str value to pd date format
start_date = pd.to_datetime(start_date) 
display(start_date) 


Timestamp('2014-08-06 00:00:00')

In [19]:
#extracting target:
st_target= recommendation['Target'].iloc[0] #value for target for given stock code
st_target

170

## generalising get_hist function:

In [20]:
recom_date = start_date.date() 
# recom_date = '2016,08,29'

recom_date

datetime.date(2014, 8, 6)

In [21]:
year = recom_date.year
month = recom_date.month
day = recom_date.day
end_year = int(year)+1
print(year)
print(month)
print(day)
print(end_year)

2014
8
6
2015


In [25]:
#getting stock history data in required format:
#get_history(symbol="CIPLA",start=date(2016,8,20),end=date(2017,8,20))
from nsepy import get_history
from datetime import date
import datetime
s_hist=get_history(symbol=stock_code,start=date(year,month,day),end=date(end_year,month,day))
s_hist.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2014-08-06,PETRONET,EQ,172.75,172.00,172.00,165.05,167.6,167.20,169.10,2305210,3.898099e+13,28092,1502732,0.6519
2014-08-07,PETRONET,EQ,167.20,167.00,170.35,166.25,170.1,169.50,169.11,748927,1.266525e+13,17678,339794,0.4537
2014-08-08,PETRONET,EQ,169.50,167.75,172.70,165.70,171.5,171.30,169.72,1094281,1.857164e+13,17241,362538,0.3313
2014-08-11,PETRONET,EQ,171.30,171.00,180.00,166.15,170.6,171.45,172.89,1104451,1.909486e+13,17902,622155,0.5633
2014-08-12,PETRONET,EQ,171.45,171.05,174.90,171.05,173.8,174.05,173.43,681679,1.182231e+13,10196,332548,0.4878


# #Creating difference column:

In [27]:
# source: https://stackoverflow.com/a/43112103
# To get the closest value

closest_value = s_hist.iloc[(s_hist['High']-st_target).abs().argsort()[:1]] #brings out the closest value row in NSE comparing the 
#recommendation target price
closest_value



,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2014-08-07,PETRONET,EQ,167.2,167.0,170.35,166.25,170.1,169.5,169.11,748927,1.266525e+13,17678,339794,0.4537


In [28]:
closest_value_p=closest_value['High']
closest_value_p  #extracting the high value in closest NSE record

Date
2014-08-07    170.35
Name: High, dtype: float64

In [29]:
recommendation

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender
1,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay


In [30]:
merged_reco = pd.concat([recommendation.reset_index(drop=True), closest_value_p.reset_index(drop=True)], axis=1)

In [31]:
merged_reco['ticker_date'] = closest_value_p.index
display(merged_reco)

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender,High,ticker_date
0,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay,170.35,2014-08-07


In [32]:
merged_reco['Recommendation_Date'] = recom_date
display(merged_reco)


,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender,High,ticker_date,Recommendation_Date
0,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay,170.35,2014-08-07,2014-08-06


In [28]:
#merged_reco['Threshold_Target'] = merged_reco['Target']*0.9 #Threshold for the target

In [33]:
merged_reco['Target_Met'] = merged_reco['High'] >= merged_reco['Target']
display(merged_reco)



,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender,High,ticker_date,Recommendation_Date,Target_Met
0,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay,170.35,2014-08-07,2014-08-06,True


In [34]:
import numpy as np

merged_reco['Days_For_Target_Met'] = merged_reco['ticker_date']- merged_reco['Recommendation_Date']
merged_reco['Days_For_Target_Met'] = np.where(merged_reco['Target_Met'], merged_reco['Days_For_Target_Met'], np.nan)
display(merged_reco)

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender,High,ticker_date,Recommendation_Date,Target_Met,Days_For_Target_Met
0,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay,170.35,2014-08-07,2014-08-06,True,1 days


In [32]:
#merged_reco['Days_For_Threshold_Target_Met'] = merged_reco['ticker_date']- merged_reco['Recommendation_Date']
#merged_reco['Days_For_Threshold_Target_Met'] = np.where(merged_reco['Threshold_Target_Met'], merged_reco['Days_For_Target_Met'], np.nan)
#display(merged_reco)

## Iterating to find the new columns to money control


In [33]:
#function to iterate overs rows:

from nsepy import get_history
from datetime import date
import datetime
import numpy as np

df = pd.DataFrame()

for index, row in money.head(10).iterrows():
       # from nsepy import get_history
       # from datetime import date
       # import datetime
       # import numpy as np
        recommendation =  money[index:index+1] #subsetting each row
        print("The index is: ", index) #printing index
        stock_code= recommendation['NSE CODE'].iloc[0] #taking the value in nse code
        start_date= recommendation['Date'].iloc[0] #taking the date value out
        start_date = start_date.replace(',', '')  #modifying as jan 15,2016 to jan 15 2016
        start_date = pd.to_datetime(start_date)   #changing to 01/15/2016
        recom_date = start_date.date() 
        year = recom_date.year #pulling year out
        month = recom_date.month #pulling month out
        day = recom_date.day # pulling day out
        end_year = int(year)+1 # passing end year as start year +1
        st_target= recommendation['Target'].iloc[0] #value for target for given stock code
        s_hist=get_history(symbol=stock_code,start=date(year,month,day),end=date(end_year,month,day)) # getting out nse history for that
        #stock between the time period
        
        #taking the ones for which s_hist has data
        if len(s_hist.index)>0:
            
            closest_value = s_hist.iloc[(s_hist['High']-st_target).abs().argsort()[:1]]  #finding out the closest stock value in the 1year period.Extracting out that row
            closest_value_p= closest_value['High'] #subsetting only high value with index
            # Trying to get the closest value as a number for comparion against target
            
            merged_reco = pd.concat([recommendation.reset_index(drop=True), closest_value_p.reset_index(drop=True)], axis=1) #concatinating two tables money dataframe and closest value
            merged_reco['ticker_date'] = np.where(closest_value['High'],closest_value.index,np.nan) #taking the index of closest value
            merged_reco['Recommendation_Date'] = recom_date

           # merged_reco['Threshold_Target'] = merged_reco['Target']*0.9 #Threshold for the target
            merged_reco['Target_Met'] = merged_reco['High'] >= merged_reco['Target']
           # merged_reco['Threshold_Target_Met'] = merged_reco['High'] >= merged_reco['Threshold_Target']
            merged_reco['Days_For_Target_Met'] = merged_reco['ticker_date']- merged_reco['Recommendation_Date']
            merged_reco['Days_For_Target_Met'] = np.where(merged_reco['Target_Met'], merged_reco['Days_For_Target_Met'], np.nan)

            #merged_reco['Days_For_Threshold_Target_Met'] = merged_reco['ticker_date']- merged_reco['Recommendation_Date']
            #merged_reco['Days_For_Threshold_Target_Met'] = np.where(merged_reco['Threshold_Target_Met'], merged_reco['Days_For_Target_Met'], np.nan)
            df = df.append(merged_reco)
   
            
    


The index is:  0
The index is:  1
The index is:  2
The index is:  3
The index is:  4
The index is:  5
The index is:  6
The index is:  7
The index is:  8
The index is:  9


In [34]:
money[50:50+1]

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender
50,611,"jul 31, 2018",petronet lng ltd,PETRONET LNG LTD,PETRONET LNG,PETRONET,buy,267,kotak securities


In [35]:
df.head()

,Sl no,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender,High,ticker_date,Recommendation_Date,Target_Met,Days_For_Target_Met
0,8144,"sep 15, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,sell,185,kotak securities,185.00,2015-03-04,2014-09-15,True,170 days
0,8571,"aug 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,170,emkay,170.35,2014-08-07,2014-08-06,True,1 days
0,9170,"jun 23, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,buy,195,icicidirect.com,194.90,2014-09-11,2014-06-23,False,NaT
0,9674,"may 14, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,hold,152,sushil finance,152.00,2014-05-27,2014-05-14,True,13 days
0,9762,"may 06, 2014",petronet lng,PETRONET LNG,PETRONET LNG,PETRONET,reduce,143,krchoksey,143.45,2014-05-14,2014-05-06,True,8 days


In [5]:
df.to_csv('11942-end.csv')

In [36]:
import os
os.getcwd()

'C:\\Users\\user\\Desktop'